# p.129 페이지 타이타닉 데이터로 분류 모델
- 전처리 : 스케일링
- 교차검증 : cross_val_score()

In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# data load

In [8]:
titanic_df = pd.read_csv('titanic_train.csv')
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# 전처리

### 결측치 처리
- Age > 평균값
- Cabin > N
- Embarked > N 

In [10]:
titanic_df['Age'] = titanic_df['Age'].fillna( titanic_df['Age'].mean() )
titanic_df['Cabin'] = titanic_df['Cabin'].fillna('N')
titanic_df['Embarked'] = titanic_df['Embarked'].fillna('N')

titanic_df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

### drop
- PassengerId  
- Name   
- Ticket            

In [12]:
titanic_df = titanic_df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,N,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,N,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,N,S


### label encoding - 범주형
- Sex
- Cabin
- Embarked

In [13]:
titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]   # 앞에 한 글자만 
titanic_df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,N,S
1,1,1,female,38.0,1,0,71.2833,C,C


In [15]:
col_names = ['Cabin', 'Sex', 'Embarked']

for col in col_names :
    le = LabelEncoder()
    le.fit(titanic_df[col])
    titanic_df[col] = le.transform(titanic_df[col])

titanic_df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,1,22.0,1,0,7.2500,7,3
1,1,1,0,38.0,1,0,71.2833,2,0


### Age, Fare - 수치형
- 그대로 사용한다 > O
- 스케일링
- 비닝

# 분할 
- train_test_split()

In [16]:
X_titanic_df = titanic_df.drop('Survived', axis=1)   # survived 빼고 
y_titanic_df = titanic_df['Survived']   # survived 만 넣기 

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=11)

# 학습 

In [20]:
# 3가지 모델 생성 
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')

In [22]:
# 학습
dt_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
lr_clf.fit(X_train, y_train)

# 예측 
dt_prd = dt_clf.predict(X_test)
rf_prd= rf_clf.predict(X_test)
lr_clf = lr_clf.predict(X_test)

# 정확도 
accuracy_score(y_test, dt_prd), accuracy_score(y_test, rf_prd), accuracy_score(y_test, lr_clf)

(0.7877094972067039, 0.8547486033519553, 0.8659217877094972)

### 교차검증

In [24]:
# 3가지 모델 생성 
dt_clf_cv = DecisionTreeClassifier(random_state=11)
rf_clf_cv = RandomForestClassifier(random_state=11)
lr_clf_cv = LogisticRegression(solver='liblinear')

In [25]:
kfold = KFold(n_splits=5)
scores = []

for train_index, test_index in kfold.split(X_titanic_df) :
    X_train, X_test = X_titanic_df.values[train_index], X_titanic_df.values[test_index]
    y_train, y_test = y_titanic_df.values[train_index], y_titanic_df.values[test_index]


    dt_clf_cv.fit(X_train, y_train)
    dt_prd = dt_clf_cv.predict(X_test)
    scores.append(accuracy_score(y_test, dt_prd))


print(scores)
print('평균 정확도: ', np.mean(scores))

    

[0.7541899441340782, 0.7808988764044944, 0.7865168539325843, 0.7696629213483146, 0.8202247191011236]
평균 정확도:  0.782298662984119


In [ ]:
# 함수로 만들기 

### 파라미터 튜닝 + 교차검증
- GridSearchCV

In [26]:
# 3가지 모델 생성 
dt_clf_gcv = DecisionTreeClassifier(random_state=11)
rf_clf_gcv = RandomForestClassifier(random_state=11)
lr_clf_gcv = LogisticRegression(solver='liblinear')

In [29]:
# decisiontree parameter 
parameters = { 'max_depth': [1,2,3], 'min_samples_split': [2,3] }

grid_df_clf = GridSearchCV(dt_clf_gcv, param_grid=parameters, scoring='accuracy', cv=5)
grid_df_clf.fit(X_train, y_train)

score_df = pd.DataFrame(grid_df_clf.cv_results_)
score_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002404,0.000801,0.002517,0.000715,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.804196,0.762238,0.825175,0.774648,0.746479,0.782547,0.028501,3
1,0.005445,0.003482,0.003837,0.001996,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.804196,0.762238,0.825175,0.774648,0.746479,0.782547,0.028501,3
2,0.004724,0.002031,0.003114,0.001431,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.748252,0.762238,0.825175,0.774648,0.746479,0.771358,0.028786,5
3,0.004847,0.001218,0.003310,0.001213,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.748252,0.762238,0.825175,0.774648,0.746479,0.771358,0.028786,5
4,0.002618,0.000989,0.001370,0.001243,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.797203,0.804196,0.818182,0.795775,0.774648,0.798001,0.014118,1
5,0.001406,0.000806,0.001003,0.000633,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.797203,0.804196,0.818182,0.795775,0.774648,0.798001,0.014118,1


In [30]:
best_dt_clf = grid_df_clf.best_estimator_
best_dt_pred = best_dt_clf.predict(X_test) # 예측치 
accuracy_score(y_test, best_dt_pred)

0.8426966292134831

# 평가

# 정확도 높이기 - 튜닝 

- 스케일링
- 속성 선택  

- 모델 : RF, LR
- 파라미터 

In [32]:
# 3가지 모델 생성 
dt_clf_scaler = DecisionTreeClassifier(random_state=11)
rf_clf_scaler = RandomForestClassifier(random_state=11)
lr_clf_scaler = LogisticRegression(solver='liblinear')

In [65]:
# 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 스케일링 후 학습, 예측, 정확도
dt_clf_scaler.fit(X_train_scaled, y_train)
rf_clf_scaler.fit(X_train_scaled, y_train)
lr_clf_scaler.fit(X_train_scaled, y_train)

dt_pred = dt_clf_scaler.predict(X_test_scaled)
rf_pred = rf_clf_scaler.predict(X_test_scaled)
lr_pred = lr_clf_scaler.predict(X_test_scaled)

print('dt_scaler:', accuracy_score(y_test, dt_pred))
print('rf_scaler:', accuracy_score(y_test, rf_pred))
print('lr_scaler:', accuracy_score(y_test, lr_pred))

dt_scaler: 0.8202247191011236
rf_scaler: 0.8539325842696629
lr_scaler: 0.8202247191011236


In [56]:
# 속성 재 선택 
titanic_df_1 = titanic_df.drop(['Fare', 'Pclass', 'Age'], axis=1)
titanic_df_1.head()

,Survived,Sex,SibSp,Parch,Cabin,Embarked
0,0,1,1,0,7,3
1,1,0,1,0,2,0
2,1,0,0,0,7,3
3,1,0,1,0,2,3
4,0,1,0,0,7,3


In [63]:
## 분할
X_titanic_df_1 = titanic_df_1.drop('Survived', axis=1)
y_titanic_df_1 = titanic_df_1['Survived']

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_titanic_df_1, y_titanic_df_1, test_size=0.2, random_state=11)

In [58]:
## 3가지 모델 생성 
dt_clf_1 = DecisionTreeClassifier(random_state=11)
rf_clf_1 = RandomForestClassifier(random_state=11)
lr_clf_1 = LogisticRegression(solver='liblinear')

## 학습
dt_clf_1.fit(X_train_1, y_train_1)
rf_clf_1.fit(X_train_1, y_train_1)
lr_clf_1.fit(X_train_1, y_train_1)

## 예측 
dt_prd_1 = dt_clf_1.predict(X_test_1)
rf_prd_1 = rf_clf_1.predict(X_test_1)
lr_clf_1 = lr_clf_1.predict(X_test_1)

## 정확도 
accuracy_score(y_test_1, dt_prd_1), accuracy_score(y_test_1, rf_prd_1), accuracy_score(y_test_1, lr_clf_1)

(0.8100558659217877, 0.7988826815642458, 0.8324022346368715)

In [81]:
# 교차검증

## 3가지 모델 생성 
dt_clf_cv_1 = DecisionTreeClassifier(random_state=11)
rf_clf_cv_1 = RandomForestClassifier(random_state=11)
lr_clf_cv_1 = LogisticRegression(solver='liblinear')

scores_cross = cross_val_score(dt_clf_cv_1, X_titanic_df_1, y_titanic_df_1, cv=5)

for i, accuracy in enumerate(scores_cross):
    print(f'교차검증 {i} 정확도(속성 재선택): {accuracy:4f}')

교차검증 0 정확도(속성 재선택): 0.776536
교차검증 1 정확도(속성 재선택): 0.803371
교차검증 2 정확도(속성 재선택): 0.780899
교차검증 3 정확도(속성 재선택): 0.775281
교차검증 4 정확도(속성 재선택): 0.825843


In [68]:
# 파라미터

## 3가지 모델 생성 
dt_clf_gcv_1 = DecisionTreeClassifier(random_state=11)
rf_clf_gcv_1 = RandomForestClassifier(random_state=11)
lr_clf_gcv_1 = LogisticRegression(solver='liblinear')

## decisiontree parameter 
parameters_1 = { 'max_depth': [1,2,3], 'min_samples_split': [2,3] }

grid_df_clf_1 = GridSearchCV(dt_clf_gcv_1, param_grid=parameters_1, scoring='accuracy', cv=5)
grid_df_clf_1.fit(X_train_1, y_train_1)

score_df_1 = pd.DataFrame(grid_df_clf_1.cv_results_)
score_df_1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005872,0.001502,0.004341,0.001418,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.755245,0.776224,0.788732,0.809859,0.746479,0.775308,0.022841,3
1,0.005156,0.002018,0.004364,0.001895,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.755245,0.776224,0.788732,0.809859,0.746479,0.775308,0.022841,3
2,0.006395,0.006793,0.001998,0.000665,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.755245,0.776224,0.788732,0.809859,0.746479,0.775308,0.022841,3
3,0.002833,0.000348,0.001978,0.000578,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.755245,0.776224,0.788732,0.809859,0.746479,0.775308,0.022841,3
4,0.003843,0.001036,0.002538,0.000419,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.769231,0.790210,0.788732,0.781690,0.760563,0.778085,0.011478,1
5,0.004523,0.001799,0.003752,0.001795,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.769231,0.790210,0.788732,0.781690,0.760563,0.778085,0.011478,1


In [74]:
best_dt_clf_1 = grid_df_clf_1.best_estimator_
best_dt_pred_1 = best_dt_clf_1.predict(X_test_1) ## 예측치 
print('GridSearchCV(속성 재선택):', accuracy_score(y_test_1, best_dt_pred_1))

GridSearchCV(속성 재선택): 0.8324022346368715
